In [31]:
import nltk 
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import json
import string
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

In [3]:
df =open('intents.json', encoding='utf-8').read()

In [4]:
data=json.loads(df)

In [30]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['hello',
    'hi',
    'hey',
    "what's up",
    'anybody here',
    'good day',
    'Evening'],
   'responses': ['Hello!',
    'Hi there, how can I help you today?',
    'Feel free to ask me anything',
    'Ask me about Data Science!']},
  {'tag': 'Target User',
   'patterns': ['Who should use this chatbot?',
    'Is this chatbot suitable for advanced users?',
    'for who they create you',
    'Who benefits the most from using this chatbot?'],
   'responses': ['This chatbot is designed for beginners and intermediate users looking to explore data science concepts.',
    'Ideal users include first-year students, entry-level professionals, and anyone starting their journey in the field.']},
  {'tag': 'thanks',
   'patterns': ['thank you',
    'thanks',
    'thanks for help',
    "that's helpful",
    'you are a greet help',
    'awesome',
    'very helpful'],
   'responses': ['Happy to help!',
    "You're welcome!",
    'Feel free to as

# First made an empty list 
## doing for loops for intents in json file then insted loops for patterns in json file
## Tokenize the pattern
## extend the pattern to put them in the empty list 
## apped the patterns in data_x to use in the training data and append tag in data_y to use it in the training
## using Lemmatizer to return the word to the root 
## then applies the lemmatizer to the word and convert it to lower and remove punctuatuion
## then put them in the set to avoid any duplicates

In [5]:
words = []
classes = []
data_x = [] 
data_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_x.append(pattern)
        data_y.append(intent["tag"])

    if intent["tag"] not in classes:
        classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))


# Initializing empty list the out_empty create a list with all zero the zeros of this list well equal the to the number of classes
## make for loops to itearte the data_x 
## if the words it's in text and it's lower append zero otherwise append zero
## then shuffling the data and put it in array beacude tensrflow just expect an array format

In [6]:
import numpy as np
import random

# Your previous code
out_empty = [0] * len(classes)
training = []

for idx, doc in enumerate(data_x):
    bow = [] 
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        if word in text:
            bow.append(1)
        else:
            bow.append(0)
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    
    training.append((bow, output_row))

random.shuffle(training)

train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

train_x = np.array(train_x)
train_y = np.array(train_y)


# the input_shape define the feature in the data set 
## dropout helps to avoid overfitting
## avtivation make the nuearal network learn the cmoplex patterns and realtionship without the activation function is gonna be like linear model
## the optimizer adam don't need to put the wieght 

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout



model = Sequential([
           #layer 1
          (Dense(128, input_shape=(len(train_x[0]),), activation="relu")),
          (Dropout(0.5)),
    #layer 2
          (Dense(64, activation="relu")),
          (Dropout(0.5)),
    #layer 3
          (Dense(len(train_y[0]), activation="softmax"))
                 ])  


adam = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
print(model.summary())



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               43136     
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 102)               6630      
                                                                 
Total params: 58022 (226.65 KB)
Trainable params: 58022 (226.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [35]:
model.layers

In [43]:
model.fit(x=train_x, y=train_y,epochs=200,verbose=1)

Epoch 1/200
13/13 [==============================] - 0s 7ms/step - loss: 0.0058 - accuracy: 0.9419
Epoch 2/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0065 - accuracy: 0.9293
Epoch 3/200
13/13 [==============================] - 0s 9ms/step - loss: 0.0058 - accuracy: 0.9495
Epoch 4/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0061 - accuracy: 0.9419
Epoch 5/200
13/13 [==============================] - 0s 7ms/step - loss: 0.0059 - accuracy: 0.9419
Epoch 6/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0057 - accuracy: 0.9470
Epoch 7/200
13/13 [==============================] - 0s 10ms/step - loss: 0.0067 - accuracy: 0.9394
Epoch 8/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0056 - accuracy: 0.9419
Epoch 9/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0059 - accuracy: 0.9192
Epoch 10/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0060 - accuracy: 0.9268
Epoch 11

In [44]:
#Preprocessing the Input
def clean_text(text):
    tokens =nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word.lower()) for word in tokens]
    return tokens
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word ==w:
                bow[idx] =1
    return np.array(bow)
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list)==0:
        result="Soory! I don't unerstand"
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"]==tag:
                result=random.choice(i["responses"])
                break
    return result

In [45]:
print("Press 0 if you don't want to chat with chatbot")
while True:
    message = input("")
    if message == "0":
        break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result.lower())
    

Press 0 if you don't want to chat with chatbot
hi
1/1 [==============================] - 0s 391ms/step
hello!
DaTa ScienCe
1/1 [==============================] - 0s 131ms/step
data science is a multidisciplinary field that involves the use of scientific methods, processes, algorithms, and systems to extract meaningful insights and knowledge from structured and unstructured data.
DATA SCIENCE
1/1 [==============================] - 0s 59ms/step
data science is a multidisciplinary field that involves the use of scientific methods, processes, algorithms, and systems to extract meaningful insights and knowledge from structured and unstructured data.
0
